# Commercial-View — Setup Guide

A step-by-step guide for **local VS Code** and **Google Colab** environments.

**What you'll get:**
- Dual environment instructions (macOS/Linux/Windows PowerShell + Colab)
- Project configuration checks
- Test & dev workflow
- Best practices and troubleshooting

> Tip: Run cells **in order**. Use the environment (Local or Colab) that suits you.

## 0) Prerequisites
- Git installed
- Python **3.11** (project uses a 3.11 venv)
- VS Code with Python & Pylance extensions (local option)
- Google account (for Colab option)

### 0.1 macOS: Installing pip (if needed)
If you encounter `pip: command not found` on macOS, use one of these methods:

#### Method 1: Using ensurepip (Recommended)
```bash
python3 -m ensurepip --upgrade
```

#### Method 2: Using Homebrew
```bash
# Install Homebrew (if not already installed)
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

# Install Python and pip
brew install python
```

#### Method 3: Using get-pip.py Script
```bash
# Download the script
curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py

# Run the script
python3 get-pip.py
```

#### Verify Installation
```bash
pip --version
# or
pip3 --version
```

## 1) Clone the repository (both environments)
If you're in **Colab**, run the Colab cell in section 4 instead.

```bash
git clone https://github.com/Jeninefer/Commercial-View.git
cd Commercial-View
git pull --rebase origin main
```

## 2) Local VS Code Setup (macOS/Linux/Windows PowerShell)

### 2.1 Create & activate virtual environment (Python 3.11)
**macOS/Linux (bash/zsh):**
```bash
python3.11 -m venv .venv
source .venv/bin/activate
python -V   # should show Python 3.11.x
```

**Windows PowerShell:**
```powershell
py -3.11 -m venv .venv
. "$PWD/.venv/Scripts/Activate.ps1"   # Windows
python -V
```

### 2.2 Upgrade pip & install dependencies
```bash
python -m pip install -U pip
python -m pip install -r requirements.txt  # if available
python -m pip install -e . || echo "(optional) dev install"
python -m pip install pytest fastapi pandas ipykernel
```

### 2.3 VS Code interpreter & Jupyter kernel
- Open Command Palette → **Python: Select Interpreter** → choose **.venv**
- Install Jupyter kernel: `python -m ipykernel install --user --name=commercial-view`

## 3) Configuration validation (local)
Run the following cells locally inside your venv to validate your setup.

In [9]:
import sys, platform
print('Python:', sys.version)
print('Executable:', sys.executable)
print('Platform:', platform.platform())

Python: 3.13.7 (v3.13.7:bcee1c32211, Aug 14 2025, 19:10:51) [Clang 16.0.0 (clang-1600.0.26.6)]
Executable: /Users/jenineferderas/Commercial-View/.venv/bin/python
Platform: macOS-15.5-arm64-arm-64bit-Mach-O


In [10]:
# Import smoke tests
modules = [
    'fastapi', 'pandas',
    'src.api', 'src.data_loader',
]
ok = True
for m in modules:
    try:
        __import__(m)
        print(f'✅ Imported {m}')
    except Exception as e:
        ok = False
        print(f'❌ Failed {m}:', e)
assert ok, 'Some imports failed — check installation.'

❌ Failed fastapi: No module named 'fastapi'
✅ Imported pandas
❌ Failed src.api: No module named 'fastapi'
✅ Imported src.data_loader


AssertionError: Some imports failed — check installation.

### 3.1 Project directory layout (expected)
```
Commercial-View/
├─ src/
│  ├─ api.py
│  └─ data_loader.py
├─ tests/
│  ├─ test_api.py
│  └─ test_data_loader.py
├─ requirements.txt (optional)
└─ setup_guide.ipynb
```

## 4) Google Colab Setup
Run the following **Colab** cell to prepare the environment. It mounts Drive (optional), clones the repo, and sets up Python packages inside the Colab runtime.

In [ ]:
# Colab: setup (uncomment Drive mount if you want to persist data)
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False
print('In Colab:', IN_COLAB)

if IN_COLAB:
    # from google.colab import drive
    # drive.mount('/content/drive')  # optional
    !git clone https://github.com/Jeninefer/Commercial-View.git || true
    %cd Commercial-View
    !git pull --rebase origin main
    %pip install -U pip
    %pip install pytest fastapi pandas uvicorn
else:
    print('This cell is intended for Colab. Skip if running locally.')

## 5) Quick API smoke test
This validates that the `/executive-summary` endpoint returns both `portfolio_overview` and `risk_indicators`.

> Note: In a notebook environment, we simply import and call the function; for a real server run `uvicorn src.api:app --reload`.

In [ ]:
try:
    from src.api import app  # type: ignore
    from fastapi.testclient import TestClient  # type: ignore
    
    client = TestClient(app)
    r = client.get('/executive-summary')
    print('Status:', r.status_code)
    print('JSON:', r.json())
    assert r.status_code == 200
    data = r.json()
    assert 'portfolio_overview' in data
    assert 'risk_indicators' in data
    print('✅ API contract OK')
except ImportError as e:
    print(f'⚠️ Import failed: {e}')
    print('Make sure you have installed the dependencies and are in the correct directory.')
except Exception as e:
    print(f'❌ Test failed: {e}')

## 6) Running tests

### Local / Colab (inside repo root)
```bash
pytest -q
```

In Colab, you can run with a `!` bang:
```python
!pytest -q
```

## 7) Dev workflow
1. Create a feature branch:
   ```bash
   git checkout -b feat/your-feature
   ```
2. Make changes, run tests locally (`pytest -q`).
3. Commit & push:
   ```bash
   git add -A
   git commit -m "feat: explain your change"
   git push -u origin feat/your-feature
   ```
4. Open a Pull Request.

## 8) Best practices
- Keep **.venv/** out of Git (use `.gitignore`).
- Pin dependencies when the project stabilizes (`requirements.txt` or `pyproject.toml`).
- Run tests before pushing.
- Use meaningful commit messages.

## 9) Troubleshooting

### Git Issues:

#### Merge Conflicts and Unmerged Files:
If you see: *"error: Pulling is not possible because you have unmerged files"*

**Quick Fix Steps:**
```bash
# 1) Check current status
git status

# 2) Remove problematic .venv files from Git (they shouldn't be tracked)
git rm -rf .venv
git add .

# 3) Clean up untracked files
rm -rf Commercial-View/  # Duplicate directory
rm -f get-pip.py        # Temporary file

# 4) Commit the cleanup
git commit -m "Remove .venv from tracking and clean up duplicates"

# 5) Now you can pull safely
git pull origin main
```

#### .gitignore Setup (Prevent Future Issues):
Create or update `.gitignore` to exclude common files:
```bash
# Create/update .gitignore
cat > .gitignore << 'EOF'
# Virtual environments
.venv/
venv/
env/

# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python

# IDE
.vscode/settings.json
.idea/

# OS
.DS_Store
Thumbs.db

# Temporary files
*.tmp
get-pip.py

# Build artifacts
build/
dist/
*.egg-info/
EOF

# Add and commit gitignore
git add .gitignore
git commit -m "Add .gitignore to prevent tracking unwanted files"
```

#### Git Network Issues:
If you encounter slow Git operations or timeouts:
```bash
# DNS cache flush (macOS)
sudo dscacheutil -flushcache
sudo killall -HUP mDNSResponder

# Test connectivity
ping -c 3 github.com

# Git network tuning
git config --global http.postBuffer 524288000
git config --global http.lowSpeedLimit 1000
git config --global http.lowSpeedTime 60

# Retry operations
git fetch origin
git pull origin main
```

#### Repository Reset (Nuclear Option):
If Git state is too corrupted, start fresh:
```bash
# Backup your work first
cp -r src/ ../Commercial-View-backup/
cp setup_guide.ipynb ../Commercial-View-backup/

# Remove and re-clone
cd ..
rm -rf Commercial-View
git clone https://github.com/Jeninefer/Commercial-View.git
cd Commercial-View

# Restore your work
cp -r ../Commercial-View-backup/src/ ./
cp ../Commercial-View-backup/setup_guide.ipynb ./

# Set up fresh environment
python3.11 -m venv .venv
source .venv/bin/activate
pip install -U pip pytest fastapi pandas ipykernel uvicorn
```

### Jupyter Kernel Issues (VS Code Notebooks):

#### "Python Environment is no longer available" Error:
If you see: *"The kernel failed to start as the Python Environment '.venv (Python 3.11.13)' is no longer available"*

**Quick Fix Steps:**
1. **Refresh Python Environments:**
   - Command Palette (`Cmd+Shift+P` / `Ctrl+Shift+P`)
   - Run: **Python: Refresh**
   - Then: **Notebook: Select Notebook Kernel**

2. **Reactivate Virtual Environment:**
   ```bash
   # Navigate to project directory
   cd Commercial-View
   
   # Reactivate virtual environment
   source .venv/bin/activate  # macOS/Linux
   # or
   .venv\Scripts\Activate.ps1  # Windows PowerShell
   
   # Verify Python path
   which python
   python --version
   ```

3. **Re-register Kernel:**
   ```bash
   # With venv activated
   python -m ipykernel install --user --name=commercial-view --display-name="Commercial View (Python 3.11)"
   ```

4. **Select Correct Kernel in VS Code:**
   - Click kernel selector (top-right of notebook)
   - Choose **"Commercial View (Python 3.11)"** or
   - Choose **"Python 3.11.x ('.venv': venv)"**

5. **If still failing, recreate environment:**
   ```bash
   # Remove old environment
   rm -rf .venv
   
   # Create fresh environment
   python3.11 -m venv .venv
   source .venv/bin/activate
   
   # Reinstall packages
   pip install -U pip
   pip install pytest fastapi pandas ipykernel uvicorn
   
   # Register kernel
   python -m ipykernel install --user --name=commercial-view --display-name="Commercial View"
   ```

#### Installation Methods:
**Using Anaconda:**
- Use Anaconda Navigator or Anaconda prompt: `conda install ipykernel`
- Or activate your environment: `conda activate your-env && conda install ipykernel`

**Using Pip (Recommended):**
```bash
# Activate your virtual environment first
source .venv/bin/activate  # macOS/Linux
# or
.venv\Scripts\Activate.ps1  # Windows PowerShell

# Install/upgrade ipykernel
pip install -U ipykernel

# Register kernel with Jupyter
python -m ipykernel install --user --name=commercial-view --display-name="Commercial View"
```

#### Version Requirements:
- **ipykernel**: version 6.0.0 or greater
- **Python**: 3.7 or greater (we use 3.11)
- **VS Code**: v1.60 or greater
- **Jupyter Extension**: v2021.9 or greater

#### Verification Steps:
```bash
# Check ipykernel version
pip show ipykernel

# List available kernels
jupyter kernelspec list

# Check VS Code version: Help → About
# Check Jupyter extension: Extensions → Search "Jupyter"
```

#### Complete Reset Process:
1. Close VS Code completely
2. Delete virtual environment: `rm -rf .venv`
3. Recreate environment: `python3.11 -m venv .venv`
4. Activate environment and install packages
5. Register kernel: `python -m ipykernel install --user --name=commercial-view`
6. Reopen VS Code
7. Select the correct kernel in your notebook

### Common Setup Issues:
- **`pip: command not found` (macOS)**: See section 0.1 for pip installation methods
- **`ImportError` for modules**: ensure the venv is active and packages are installed
- **`pytest` not found**: `python -m pip install pytest` inside the venv/Colab
- **Path issues on macOS**: prefer absolute paths; resolve with `Path(...).resolve()`
- **FastAPI endpoint missing keys**: confirm `src/api.py` returns both `portfolio_overview` and `risk_indicators` keys
- **PowerShell vs bash**: Use `source .venv/bin/activate` on macOS/Linux, `. "./.venv/Scripts/Activate.ps1"` on Windows PowerShell

### Python Version Issues:
- **Wrong Python version**: Use `python3.11` explicitly or install via Homebrew/pyenv
- **Multiple Python installations**: Use `which python3.11` to verify the correct path
- **Virtual environment not working**: Try `python3.11 -m venv --clear .venv` to recreate

### macOS Specific:
- **Command Line Tools**: Install with `xcode-select --install`
- **Homebrew path issues**: Add `/opt/homebrew/bin` to your PATH for Apple Silicon Macs
- **Permission errors**: Avoid using `sudo` with pip; use virtual environments instead

### Still Having Issues?
- Check [IPython documentation](https://ipython.readthedocs.io/en/stable/install/kernel_install.html) for detailed kernel installation
- Log a bug report with VS Code Jupyter extension if problems persist
- Ensure all components are up to date

---
© Commercial-View — Setup guide generated 2025-01-27